In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
# loading the directories 
training_dir = '../input/cabbagetypescnn/cabbage'
#validation_dir = '../input/fruits/fruits-360_dataset/fruits-360/Test/'
#test_dir = '../input/fruits/fruits-360_dataset/fruits-360/test-multiple_fruits/'

In [3]:
# useful for getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
#valid_image_files = glob(validation_dir + '/*/*.jp*g')

In [4]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 4


In [5]:
#train_df, validate_df = train_test_split(training_dir, test_size=0.20, random_state=0) 

In [6]:
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [150, 150]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-07-18 14:50:52.297071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-18 14:50:52.404698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-18 14:50:52.405494: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-18 14:50:52.406739: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 0s 0us/step


In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [8]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=IMAGE_SIZE,
    batch_size=200,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    training_dir, # same directory as training data
    target_size=IMAGE_SIZE,
    batch_size=200,
    class_mode='categorical',
    subset='validation') # set as validation data

#model.fit_generator(
#    train_generator,
#    steps_per_epoch = train_generator.samples // 200,
#    validation_data = validation_generator, 
#    validation_steps = validation_generator.samples // 200,
 #   epochs = nb_epochs)

Found 82 images belonging to 4 classes.
Found 20 images belonging to 4 classes.


In [9]:
# Image Augmentation

#from keras.preprocessing.image import ImageDataGenerator
#from keras.applications.vgg16 import preprocess_input

#training_datagen = ImageDataGenerator(
 #                                   rescale=1./255,   # all pixel values will be between 0 an 1
 #                                   shear_range=0.2, 
  #                                  zoom_range=0.2,
  #                                  horizontal_flip=True,
   #                                 preprocessing_function=preprocess_input)

#validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

#training_generator = training_datagen.flow_from_directory(train_df, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')
#validation_generator = validation_datagen.flow_from_directory(validate_df, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')

In [10]:
# The labels are stored in class_indices in dictionary form. 
# checking the labels
train_generator.class_indices

{'bokchoy': 0, 'napa': 1, 'red': 2, 'savoy': 3}

In [11]:
#training_images = 82
#validation_images = 20

history = model.fit_generator(train_generator,
  #                 steps_per_epoch = 82,  # this should be equal to total number of images in training set. But to speed up the execution, I am only using 10000 images. Change this for better results. 
                   epochs = 8,  # change this for better results
                   validation_data = validation_generator)
      #             validation_steps = 20)  # this should be equal to total number of images in validation set.

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
2022-07-18 14:50:59.047495: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/8


2022-07-18 14:51:02.250719: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1/1 [==============================] - 14s 14s/step - loss: 1.4402 - accuracy: 0.3780 - val_loss: 1.2190 - val_accuracy: 0.4500
Epoch 2/8
1/1 [==============================] - 2s 2s/step - loss: 1.2439 - accuracy: 0.3659 - val_loss: 0.9797 - val_accuracy: 0.6500
Epoch 3/8
1/1 [==============================] - 2s 2s/step - loss: 1.0342 - accuracy: 0.5610 - val_loss: 0.8375 - val_accuracy: 0.6000
Epoch 4/8
1/1 [==============================] - 3s 3s/step - loss: 0.8414 - accuracy: 0.7195 - val_loss: 0.7485 - val_accuracy: 0.8000
Epoch 5/8
1/1 [==============================] - 2s 2s/step - loss: 0.6878 - accuracy: 0.8049 - val_loss: 0.6609 - val_accuracy: 0.8500
Epoch 6/8
1/1 [==============================] - 2s 2s/step - loss: 0.5863 - accuracy: 0.8902 - val_loss: 0.5541 - val_accuracy: 0.9000
Epoch 7/8
1/1 [==============================] - 2s 2s/step - loss: 0.4911 - accuracy: 0.9146 - val_loss: 0.5570 - val_accuracy: 0.9000
Epoch 8/8
1/1 [==============================] - 3s 3s/s

In [12]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))

Training Accuracy = [0.37804877758026123, 0.3658536672592163, 0.5609756112098694, 0.7195122241973877, 0.8048780560493469, 0.8902438879013062, 0.9146341681480408, 0.9634146094322205]
Validation Accuracy = [0.44999998807907104, 0.6499999761581421, 0.6000000238418579, 0.800000011920929, 0.8500000238418579, 0.8999999761581421, 0.8999999761581421, 0.949999988079071]
